In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import os
os.chdir('/content/gdrive/MyDrive/final_project')
!pwd
!ls

/content/gdrive/MyDrive/final_project
mydataset_v4.json  mydataset_v6.json  mydataset_v7.json  mymodel_2  training.ipynb


checking module existence

In [3]:
!pip show evaluate
!pip show datasets
!pip show torch
#if any module missing here , run !pip install [module]

Name: evaluate
Version: 0.4.2
Summary: HuggingFace community-driven open-source library of evaluation
Home-page: https://github.com/huggingface/evaluate
Author: HuggingFace Inc.
Author-email: leandro@huggingface.co
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: datasets, dill, fsspec, huggingface-hub, multiprocess, numpy, packaging, pandas, requests, tqdm, xxhash
Required-by: 
Name: datasets
Version: 2.19.2
Summary: HuggingFace community-driven open-source library of datasets
Home-page: https://github.com/huggingface/datasets
Author: HuggingFace Inc.
Author-email: thomas@huggingface.co
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, dill, filelock, fsspec, huggingface-hub, multiprocess, numpy, packaging, pandas, pyarrow, pyarrow-hotfix, pyyaml, requests, tqdm, xxhash
Required-by: evaluate
Name: torch
Version: 2.3.0+cu121
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page

In [4]:
# !pip install evaluate
# !pip install datasets

import important files

In [5]:
import transformers
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn.functional as F
import numpy as np
import pandas as pd
import os
import warnings
warnings.simplefilter("ignore")

Load our dataset

In [6]:
from datasets import load_dataset

# Load a JSON file
dd1 = load_dataset('json', data_files='mydataset_v7.json',field = 'train')['train']
dd2 = load_dataset('json', data_files='mydataset_v7.json',field ='validation')['train']
dataset = {'train':dd1,'validation':dd2}

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
dataset

{'train': Dataset({
     features: ['id', 'answers', 'question', 'context'],
     num_rows: 750
 }),
 'validation': Dataset({
     features: ['id', 'answers', 'question', 'context'],
     num_rows: 150
 })}

print the first slice of the dataset

In [8]:
print(dataset['train'][0]['context'])
print(dataset['train'][0]['question'])
print(dataset['train'][0]['answers']['text'][0])
print(dataset['train'][0]['answers']['answer_start'][0])
print('')

print(dataset['validation'][0]['context'])
print(dataset['validation'][0]['question'])
print(dataset['validation'][0]['answers']['text'][0])
print(dataset['validation'][0]['answers']['answer_start'][0])

Write an email from James to Professor Martinez regarding a missed assignment deadline.
who is writing this letter
James
20

Write an email from Charlotte to classmate Ethan about exchanging research articles.
who is writing this letter
Charlotte
20


train data processer


In [9]:
from transformers import AutoTokenizer
from time import sleep
#del tokenizer
trained_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(trained_checkpoint)

def train_data_preprocess(examples):

    """
    generate start and end indexes of answer in context
    """

    def find_context_start_end_index(sequence_ids):
        """
        returns the token index in whih context starts and ends
        """
        token_idx = 0
        while sequence_ids[token_idx] != 1:  #means its special tokens or tokens of queston
            token_idx += 1                   # loop only break when context starts in tokens
        context_start_idx = token_idx

        while sequence_ids[token_idx] == 1:
            token_idx += 1
        context_end_idx = token_idx - 1
        return context_start_idx,context_end_idx

    print(examples["question"])
    questions = [q.strip() for q in examples["question"]]
    context = examples["context"]
    answers = examples["answers"]
    inputs = tokenizer(
        questions,
        context,
        max_length=512,
        truncation="only_second",
        stride=128,
        return_overflowing_tokens=True,  #returns id of base context
        return_offsets_mapping=True,  # returns (start_index,end_index) of each token
        padding="max_length"
    )


    start_positions = []
    end_positions = []

    for i,mapping_idx_pairs in enumerate(inputs['offset_mapping']):
        context_idx = inputs['overflow_to_sample_mapping'][i]

        # from main context
        answer = answers[context_idx]
        answer_start_char_idx = answer['answer_start'][0]
        answer_end_char_idx = answer_start_char_idx + len(answer['text'][0])


        # now we have to find it in sub contexts
        tokens = inputs['input_ids'][i]
        sequence_ids = inputs.sequence_ids(i)

        # finding the context start and end indexes wrt sub context tokens
        context_start_idx,context_end_idx = find_context_start_end_index(sequence_ids)

        #if the answer is not fully inside context label it as (0,0)
        # starting and end index of charecter of full context text
        context_start_char_index = mapping_idx_pairs[context_start_idx][0]
        context_end_char_index = mapping_idx_pairs[context_end_idx][1]


        #If the answer is not fully inside the context, label is (0, 0)
        if (context_start_char_index > answer_start_char_idx) or (
            context_end_char_index < answer_end_char_idx):
            start_positions.append(0)
            end_positions.append(0)

        else:

            # else its start and end token positions
            # here idx indicates index of token
            idx = context_start_idx
            while idx <= context_end_idx and mapping_idx_pairs[idx][0] <= answer_start_char_idx:
                idx += 1
            start_positions.append(idx - 1)


            idx = context_end_idx
            while idx >= context_start_idx and mapping_idx_pairs[idx][1] > answer_end_char_idx:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

train_sample = dataset["train"]

train_dataset = train_sample.map(
    train_data_preprocess,
    batched=True,
    remove_columns=dataset["train"].column_names
)

len(dataset["train"]),len(train_dataset)

Map:   0%|          | 0/750 [00:00<?, ? examples/s]

['who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is g

(750, 750)

evaluation set preprosessor

In [10]:
from transformers import AutoTokenizer

def preprocess_validation_examples(examples):
    """
    preprocessing validation data
    """
    print(examples["question"])
    questions = [q.strip() for q in examples["question"]]

    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=512,
        truncation="only_second",
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")

    base_ids = []

    for i in range(len(inputs["input_ids"])):

        # take the base id (ie in cases of overflow happens we get base id)
        base_context_idx = sample_map[i]
        base_ids.append(examples["id"][base_context_idx])

        # sequence id indicates the input. 0 for first input and 1 for second input
        # and None for special tokens by default
        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        # for Question tokens provide offset_mapping as None
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["base_id"] = base_ids
    return inputs


del tokenizer
trained_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(trained_checkpoint)
#put evaluation set here
data_val_sample = dataset["validation"].select([i for i in range(100)])
eval_set = data_val_sample.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=dataset["validation"].column_names,
)
len(eval_set)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

['who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is g

100

If we take the theoritical answers and predicted answers,the number of shared words between theoritical and predicted answer is the basis for f1 score.precision is the ratio of the number of shared words to the total number of words in the prediction, and recall is the ratio of the number of shared words to the total number of words in the ground truth.


evaluation metric

In [11]:
import numpy as np
import collections
import evaluate

def predict_answers_and_evaluate(start_logits,end_logits,eval_set,examples):
    """
    make predictions
    Args:
    start_logits : strat_position prediction logits
    end_logits: end_position prediction logits
    eval_set: processed val data
    examples: unprocessed val data with context text
    """
    # appending all id's corresponding to the base context id
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(eval_set):
        example_to_features[feature["base_id"]].append(idx)

    n_best = 20
    max_answer_length = 30
    predicted_answers = []

    for example in examples:
        example_id = example["id"]
        context = example["context"]
        answers = []

        # looping through each sub contexts corresponding to a context and finding
        # answers
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = eval_set["offset_mapping"][feature_index]

            # sorting the predictions of all hidden states and taking best n_best prediction
            # means taking the index of top 20 tokens
            start_indexes = np.argsort(start_logit).tolist()[::-1][:n_best]
            end_indexes = np.argsort(end_logit).tolist()[::-1][:n_best]


            for start_index in start_indexes:
                for end_index in end_indexes:

                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length.
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                       ):
                        continue

                    answers.append({
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                        })


            # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    metric = evaluate.load("squad")

    theoretical_answers = [
            {"id": ex["id"], "answers": ex["answers"]} for ex in examples
    ]

    metric_ = metric.compute(predictions=predicted_answers, references=theoretical_answers)
    return predicted_answers,metric_

train part


dataloader

In [12]:
from torch.utils.data import DataLoader, Dataset


class DataQA(Dataset):
    def __init__(self, dataset,mode="train"):
        self.mode = mode


        if self.mode == "train":
            # sampling
            self.dataset = dataset["train"]
            self.data = self.dataset.map(train_data_preprocess, batched = True,remove_columns= dataset["train"].column_names,)
        else:
            self.dataset = dataset["validation"]
            self.data = self.dataset.map(preprocess_validation_examples,batched=True,remove_columns = dataset["validation"].column_names,)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):

        out = {}
        example = self.data[idx]
        out['input_ids'] = torch.tensor(example['input_ids'])
        out['attention_mask'] = torch.tensor(example['attention_mask'])


        if self.mode == "train":

            out['start_positions'] = torch.unsqueeze(torch.tensor(example['start_positions']),dim=0)
            out['end_positions'] = torch.unsqueeze(torch.tensor(example['end_positions']),dim=0)

        return out

prepare dataset


In [13]:
trained_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(trained_checkpoint)

val_dataset = DataQA(dataset,mode="validation")
train_dataset = DataQA(dataset,mode="train")





for i,d in enumerate(train_dataset):
    for k in d.keys():
        print(k + ' : ', d[k].shape)
    print('--'*40)

    if i == 3:
        break

print('__'*50)
for i,d in enumerate(val_dataset):
    for k in d.keys():
        print(k + ' : ', len(d[k]))
    print('--'*40)

    if i == 3:
        break

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

['who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is g

Map:   0%|          | 0/750 [00:00<?, ? examples/s]

['who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is g

prepare dataloader


In [14]:
from transformers import default_data_collator
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=default_data_collator,
    batch_size=2,
)
eval_dataloader = DataLoader(
    val_dataset, collate_fn=default_data_collator, batch_size=2
)




for batch in train_dataloader:
   print(batch['input_ids'].shape)
   print(batch['attention_mask'].shape)
   print(batch['start_positions'].shape)
   print(batch['end_positions'].shape)
   break

print('---'*20)

for batch in eval_dataloader:
   print(batch['input_ids'].shape)
   print(batch['attention_mask'].shape)
   break

torch.Size([2, 512])
torch.Size([2, 512])
torch.Size([2, 1])
torch.Size([2, 1])
------------------------------------------------------------
torch.Size([2, 512])
torch.Size([2, 512])


prepare model and device

In [15]:
from transformers import DistilBertForQuestionAnswering
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Available device: {device}')

checkpoint =  "distilbert-base-uncased"
model = DistilBertForQuestionAnswering.from_pretrained(checkpoint)
model = model.to(device)

Available device: cuda


Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
from transformers import AdamW
from tqdm.notebook import tqdm
import datetime
import numpy as np
import collections
import evaluate

optimizer = AdamW(model.parameters(), lr=2e-5)

epochs = 10

# Total number of training steps is [number of batches] x [number of epochs].
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs
print(total_steps)


def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

3750


In [17]:
# we need processed validation data to get offsets at the time of evaluation
validation_processed_dataset = dataset["validation"].map(preprocess_validation_examples,
            batched=True,remove_columns = dataset["validation"].column_names,
               )

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

['who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is going to receive this letter', 'what is the email about', 'who is writing this letter', 'who is g

training ,testing and evaluating

In [18]:
import random,time
import numpy as np

# to reproduce results
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


#storing all training and validation stats
stats = []


#to measure total training time
total_train_time_start = time.time()

for epoch in range(epochs):
    print(' ')
    print(f'=====Epoch {epoch + 1}=====')
    print('Training....')

    # ===============================
    #    Train
    # ===============================
    # measure how long training epoch takes
    t0 = time.time()

    training_loss = 0
    # loop through train data
    model.train()


    for step,batch in enumerate(train_dataloader):

        # we will print train time in every 40 epochs
        if step%40 == 0 and not step == 0:
              elapsed_time = format_time(time.time() - t0)
              # Report progress.
              print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed_time))



        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)



        #set gradients to zero
        model.zero_grad()

        result = model(input_ids = input_ids,
                        attention_mask = attention_mask,
                        start_positions = start_positions,
                        end_positions = end_positions,
                        return_dict=True)

        loss = result.loss

        #accumulate the loss over batches so that we can calculate avg loss at the end
        training_loss += loss.item()

        #perform backward prorpogation
        loss.backward()

        # update the gradients
        optimizer.step()

    # calculate avg loss
    avg_train_loss = training_loss/len(train_dataloader)

    # calculates training time
    training_time = format_time(time.time() - t0)


    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))


    # ===============================
    #    Validation
    # ===============================

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()


    start_logits,end_logits = [],[]
    for step,batch in enumerate(eval_dataloader):


        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)


        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():
             result = model(input_ids = input_ids,
                        attention_mask = attention_mask,return_dict=True)



        start_logits.append(result.start_logits.cpu().numpy())
        end_logits.append(result.end_logits.cpu().numpy())


    start_logits = np.concatenate(start_logits)
    end_logits = np.concatenate(end_logits)
    # start_logits = start_logits[: len(val_dataset)]
    # end_logits = end_logits[: len(val_dataset)]




    # calculating metrics
    answers,metrics_ = predict_answers_and_evaluate(start_logits,end_logits,validation_processed_dataset,dataset["validation"])
    print(f'Exact match: {metrics_["exact_match"]}, F1 score: {metrics_["f1"]}')


    print('')
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)

    print("  Validation took: {:}".format(validation_time))

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_train_time_start)))

 
=====Epoch 1=====
Training....
  Batch    40  of    375.    Elapsed: 0:00:07.
  Batch    80  of    375.    Elapsed: 0:00:11.
  Batch   120  of    375.    Elapsed: 0:00:16.
  Batch   160  of    375.    Elapsed: 0:00:21.
  Batch   200  of    375.    Elapsed: 0:00:25.
  Batch   240  of    375.    Elapsed: 0:00:30.
  Batch   280  of    375.    Elapsed: 0:00:34.
  Batch   320  of    375.    Elapsed: 0:00:39.
  Batch   360  of    375.    Elapsed: 0:00:43.

  Average training loss: 0.72
  Training epoch took: 0:00:45

Running Validation...
Exact match: 32.666666666666664, F1 score: 82.23174603174604

  Validation took: 0:00:19
 
=====Epoch 2=====
Training....
  Batch    40  of    375.    Elapsed: 0:00:04.
  Batch    80  of    375.    Elapsed: 0:00:09.
  Batch   120  of    375.    Elapsed: 0:00:13.
  Batch   160  of    375.    Elapsed: 0:00:18.
  Batch   200  of    375.    Elapsed: 0:00:22.
  Batch   240  of    375.    Elapsed: 0:00:27.
  Batch   280  of    375.    Elapsed: 0:00:32.
  Batch 

save model

In [19]:
import torch
torch.save(model.state_dict(), 'mymodel_2') #change to our own path


testing the saved model

In [ ]:
import torch
from transformers import DistilBertForQuestionAnswering
from transformers import AutoTokenizer
import warnings
warnings.simplefilter("ignore")

del tokenizer
del model
weight_path = "mymodels_2"
# # loading tokenizer
trained_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(trained_checkpoint)
# #loading the model

model = DistilBertForQuestionAnswering.from_pretrained(trained_checkpoint)
model.load_state_dict(torch.load('mymodel_2'))
model.eval()

In [24]:


question ='what is the email about'
context = 'Draft an email about clarification on a lecture topic from Sarah to TA John asking. the lecture is present on Monday'

inputs = tokenizer(question, context, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
start_logits,end_logits = outputs['start_logits'],outputs['end_logits']
# Find the tokens with the highest `start` and `end` scores.
answer_start = torch.argmax(start_logits)
answer_end = torch.argmax(end_logits)
predict_answer_tokens = inputs.input_ids[0, answer_start : answer_end + 1]
tokenizer.decode(predict_answer_tokens)


'clarification on a lecture topic from sarah to ta john asking.'